In [20]:
import requests
import msal
from powerbiclient import Report
from powerbiclient.authentication import DeviceCodeLoginAuthentication

In [21]:
# Auth object for demo
auth = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code LTLMME4JG to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [22]:
report = Report(group_id='ceb0d1d0-6226-4aef-b245-fac7f89bb52e', report_id='025bdeaf-cf4e-40b9-a4c6-01446cbdff3f', auth=auth)

In [23]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

In [24]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

In [25]:
# Render report
report

Report()

In [26]:
# Get pages
def get_active_page(self):
    
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']
visuals = report.visuals_on_page(active_page_name)
#export visuals var in json format

#give me only the keys 'title' and 'type' from the visuals
#visuals = [{k: v for k, v in visual.items() if k in ['title', 'type']} for visual in visuals]
visuals

In [19]:
# Get visual whose type is 'clusteredColumnChart'
visual = next(filter(lambda visual: visual['type'] == 'barChart', visuals))
visual_name = visual['name']

summarized_exported_data = report.export_visual_data(active_page_name,
                                                 visual_name, rows=20)
summarized_exported_data

Sales,Product
$44874921.10,Tailspin Heli - Max Pro Flight - 6ch
$9530960.15,6CCP-A Helicopter
$7728618.80,Piper Cub 4 Channel
$5450602.05,P47 5 Channel
$1605862.40,Tailspin Heli - Co-Ax Pro Mk I - 4ch
$1455897.75,Tailspin Aviator Mk2-15
$1453235.60,Tailspin Warbird BM32
$1284627.85,3CAX-B Helicopter
$1225510.80,Tailspin Aviator Mk2-12
$949895.65,P51
$947815.10,SkyTrainer
$499292.15,P47 4 Channel
$490673.60,Piper Cub 3 Channel
$472215.30,Trainer - Tailspin GL-155
$206465.15,4CAX-B Helicopter
$183147.55,Trainer - Tailspin GL-120
$173362.95,3CFP-I Helicopter
$119821.15,Tailspin Aviator Mk2-11

